# Imports & Setup

In [27]:
from IPython.display import clear_output

!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip /content/nature_12K.zip

clear_output()

In [40]:
from PIL import Image

import os
from glob import glob
import time


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary
from torch.optim import Adam 

# Dataloader

- Dataset Class for Setting up the data loading process
- Sections to fill in this script: `_init_transform()`

In [41]:
class inaturalist(Dataset):
    def __init__(self, root_dir, mode = 'train', transform = True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform      
        self._init_dataset()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.labels = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, 'train')))
        if self.mode == 'train': 
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'train', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        elif self.mode == 'val':
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, 'val', dirs[dir], '*.jpg')))
                self.labels += [dir]*len(files)            
                self.files += files
        else:
            print("No Such Dataset Mode")
            return None
    
    def _init_transform(self):
        self.transform = transforms.Compose([
             transforms.Resize((256,256)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30), 
            transforms.ToTensor(),
            transforms.Normalize((0.5),(0.5))
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
       #     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
            # Useful link for this part: https://pytorch.org/vision/stable/transforms.html
            #----------------YOUR CODE HERE---------------------#
        ])
    
    def __getitem__(self, index):
        img = Image.open(self.files[index]).convert('RGB')
        label = self.labels[index]

        if self.transforms:
            img = self.transform(img)

        label = torch.tensor(label, dtype = torch.long)

        return img, label

    def __len__(self):
        return len(self.files)

# Model

- Class to define the model which we will use for training
- Stuff to fill in: The Architecture of your model, the `forward` function to define the forward pass

NOTE!: You are NOT allowed to use pretrained models for this task

In [42]:
# VGG_19 = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

# 2 convolutional layers with max pooling and relu activation
# then 3 linear Deep layers

class Classifier(nn.Module):
    def __init__(self, n_classes):
          super(Classifier, self).__init__()
          self.conv1 = nn.Conv2d(in_channels=3,
                               out_channels=6,
                               kernel_size=5)

          self.conv2 = nn.Conv2d(in_channels=6,
                               out_channels=16,
                               kernel_size=5)

          self.fc_1 = nn.Linear(16 * 61 * 61, 120)
          self.fc_2 = nn.Linear(120, 84)
          self.fc_3 = nn.Linear(84, n_classes)


        # Useful Link: https://pytorch.org/docs/stable/nn.html
        #------------ENTER YOUR MODEL HERE----------------#        

    def forward(self, x):
        #---------Assuming x to be the input to the model, define the forward pass-----------#
        #(3, 256, 256) ---> input
        x = self.conv1(x) 
        #(6, 252, 252) ---> output
        x = F.max_pool2d(x, kernel_size=2)
        #(6, 126, 126)
        x = F.relu(x)
        x = self.conv2(x)
        #(16, 122, 122)
        x = F.max_pool2d(x, kernel_size=2)
        #(16, 61, 61)
        x = F.relu(x)
        x = x.view(x.shape[0], -1)
        h = x
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        x = F.relu(x)
        x = self.fc_3(x)
        return x, h

# Training

- Sections to Fill: Define `loss` function, `optimizer` and model, `train` and `eval` functions and the training loop


## Hyperparameters

Feel free to change these hyperparams based on your machine's capactiy

In [43]:
batch_size = 32
epochs = 5
learning_rate = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

In [60]:
trainset = inaturalist(root_dir='inaturalist_12K', mode='train')
valset = inaturalist(root_dir='inaturalist_12K', mode = 'val')

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = DataLoader(valset, batch_size=1, shuffle=False, num_workers=4)

OUTPUT_DIM = 15
model = Classifier(OUTPUT_DIM)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,158,751 trainable parameters


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Loss Function and Optimizer

In [61]:
# USEFUL LINK: https://pytorch.org/docs/stable/nn.html#loss-functions
#---Define the loss function to use, model object and the optimizer for training---#

model = model.to(device)

optimizer = Adam(model.parameters(), learning_rate)
loss_fun = F.cross_entropy

## Checkpoints

To save your model weights

In [62]:
checkpoint_dir = 'checkpoints'
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

## Utility Functions

In [63]:
def get_model_summary(model, input_tensor_shape):
    summary(model, input_tensor_shape)

def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct/total

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Train

In [64]:
#def train(model, dataset, optimizer, criterion, device):
    #------YOUR CODE HERE-----#


def train(model, epochs, trainloader):
    model.train()
    total_num = len(trainloader.dataset)
    train_loss = 0
    correct_num = 0

    for image, label in trainloader:
        image = image.to(device)
        label = label.to(device)
        # Convert the tag from int32 type to long type, otherwise the calculation loss will report an error
        label = label.to(torch.long)

        output,_ = model(image)
        loss = loss_fun(output, label)
        train_loss += loss.item() * label.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = torch.argmax(output, dim=-1)
        correct_num += label.eq(predict).sum()

    train_loss = train_loss / total_num
    train_acc = correct_num / total_num
    print('epoch: {} --> train_loss: {:.6f} - train_acc: {:.6f} - '.format(
        epoch, train_loss, train_acc), end='')


## Eval

In [69]:
#def eval(model, dataset, criterion, device):

    #------YOUR CODE HERE-----#

def evaluate(model, eval_ds, mode='val'):
    model.eval()

    total_num = len(eval_ds.dataset)
    eval_loss = 0
    correct_num = 0

    for image, label in eval_ds:
            image = image.to(device)
            label = label.to(device)
            label = label.to(torch.long)
            
            output,_ = model(image)
        
            loss = loss_fun(output, label)
            eval_loss += loss.item() * label.size(0)

            predict = torch.argmax(output, dim=-1)
            correct_num += label.eq(predict).sum()
        
            eval_loss = eval_loss / total_num
            eval_acc = correct_num / total_num
    
    print('{}_loss: {:.6f} - {}_acc: {:.6f}'.format(
        mode, eval_loss, mode, eval_acc))

## Training

In [66]:
for epoch in range(5):
    train(model, epoch, trainloader)
    evaluate(model, valloader)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: 0 --> train_loss: 2.204563 - train_acc: 0.206721 - val_loss: 0.001023 - val_acc: 0.168000
epoch: 1 --> train_loss: 2.064819 - train_acc: 0.268527 - val_loss: 0.000665 - val_acc: 0.200000
epoch: 2 --> train_loss: 2.002788 - train_acc: 0.292629 - val_loss: 0.001123 - val_acc: 0.168000
epoch: 3 --> train_loss: 1.950952 - train_acc: 0.314831 - val_loss: 0.000696 - val_acc: 0.245000
epoch: 4 --> train_loss: 1.913288 - train_acc: 0.328533 - val_loss: 0.000896 - val_acc: 0.216500


In [70]:
evaluate(model, trainloader, mode='test')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


test_loss: 0.002529 - test_acc: 0.346735
